In [1]:
from ibl_pipeline import behavior as behavior_analyses
from ibl_pipeline import behavior as behavior
from ibl_pipeline import subject, acquisition, action
from ibl_pipeline.utils import psychofit as psy
import numpy as np
import datetime
import datajoint as dj
import json
import pandas as pd

Connecting shan@datajoint.internationalbrainlab.org:3306


In [2]:
import plotly
import plotly.graph_objs as go
import statsmodels.stats.proportion as smp
plotly.offline.init_notebook_mode(connected=True)

In [3]:
subj = subject.Subject & 'subject_nickname="CSHL_006"'
# get water and date
water_info_query = (action.WaterAdministration & subj).proj(
    'water_administered', 'watertype_name', water_date='DATE(administration_time)')
water_info = water_info_query.fetch(as_dict=True)
water_info = pd.DataFrame(water_info)

In [7]:
water_types = water_info.watertype_name.unique()

In [8]:
water_types

array(['Water', 'Water 10% Sucrose', 'Water 2% Citric Acid'], dtype=object)

In [9]:
water_info.pop('administration_time')
water_info.pop('subject_uuid');

In [10]:
water_info_type = water_info.pivot_table(
    index='water_date', columns='watertype_name', values='water_administered', aggfunc='sum')

In [11]:
weight_info_query = (action.Weighing & subj).proj(
    'weight', weighing_date='DATE(weighing_time)')

In [12]:
weight_info = weight_info_query.fetch(as_dict=True)
weight_info = pd.DataFrame(weight_info)

In [13]:
weight_info.pop('subject_uuid');

In [14]:
weight_info.pop('weighing_time');

In [15]:
weight_info.pivot_table(index='weighing_date', values='weight')

,weight
weighing_date,
2019-02-19,22.375
2019-02-20,19.940
2019-02-21,20.000
2019-02-22,18.410
2019-02-23,18.470
2019-02-26,18.650
2019-02-28,19.620
2019-03-01,19.900
2019-03-04,20.310


In [16]:
data = [
    go.Bar(
        x = [t.strftime('%Y-%m-%d') for t in water_info_type.index.tolist()],
        y = water_info_type[water_type].tolist(),
        name = water_type,
        yaxis = 'y1')
    for water_type in water_types
]

data.append(
    go.Scatter(
        x = [t.strftime('%Y-%m-%d') for t in weight_info['weighing_date'].tolist()],
        y = weight_info['weight'].tolist(),
        mode = 'lines+markers',
        name = 'Weight',
        marker = dict(
            size = 6,
            color = 'black'),
        yaxis = 'y2'
    ))

layout = go.Layout(
    yaxis=dict(
        title='Water intake (mL)',
    ),
    yaxis2=dict(
        title='Weight (g)',
        overlaying = 'y',
        side='right'
    ),
    width=600,
    height=400,
    title='Water intake and weight', 
    xaxis={'title':'Date'}, 
    legend=dict(
        x=0, 
        y=1.2,
        orientation='h')
)

In [17]:
fig = go.Figure(data=data, layout=layout)

In [18]:
plotly.offline.iplot(fig)

In [ ]:
f = open("water_weight.json","w")
s = json.dumps(fig.to_plotly_json())
f.write(s)
f.close()

In [ ]:
key_source = subject.Subject.aggr(
        action.Weighing * action.WaterAdministration,
        last_date='DATE(GREATEST(MAX(weighing_time), MAX(administration_time)))'
    )